# 1. Introduction

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>1.1 Background</b></p>
</div>

Welcome to this competition on predicting **Cervical Spine Fractures** from **Computed Tomography (CT) scans**. The **motivation** to use AI for this task is that a **quick** diagnosis can **reduce** the chance of **neurologic deterioration** and paralysis after trauma.

<center>
<img src="https://www.holisticbodyworks.com.au/wp-content/uploads/2018/05/Thoracic-Spine.jpg" width=400>
</center>

**Dataset origin**

The dataset we are using is made up of roughly **3000 CT studies**, [from twelve locations and across six continents](https://www.kaggle.com/competitions/rsna-2022-cervical-spine-fracture-detection/overview/acknowledgements). Spine **radiology specialists** have provided **annotations** to indicate the presence, vertebral level and location of any cervical spine fractures.

Special thanks to the competition **hosts** for providing such a comprehensive dataset:
* *Radiological Society of North America (RSNA)*
* *American Society of Neuroradiology (ASNR)*
* *American Society of Spine Radiology (ASSR)*

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>1.2 Evaluation metric</b></p>
</div>

We need to predict the **probability of fracture** for each of the **seven cervical vertebrae** denoted by C1, C2, C3, C4, C5, C6 and C7 as well as an **overall probability** of any fractures in the cervical spine. This means there will be **8 rows per image id** in the submission file. Note that fractures in the skull base, thoracic spine, ribs, and clavicles are **ignored**.
    
The **competition metric** is a **weighted multi-label logarithmic loss** (averaged across all patients)
    
$$
L_{ij} = - w_j \left(y_{ij} \log(p_{ij}) + (1-y_{ij}) \log(1-p_{ij})  \right)
$$

where the **weights** [are given by](https://www.kaggle.com/competitions/rsna-2022-cervical-spine-fracture-detection/discussion/340392)
    
$$
w_{j} = \begin{cases}
1, \qquad \text{if vertebrae negative} \\
2, \qquad \text{if vertebrae positive} \\
7, \qquad \text{if patient negative} \\
14, \qquad \text{if patient positive}
\end{cases}
$$

<br>
<center>
<img src="https://i.postimg.cc/wBYCYqFG/metric-plot.png" width=600>
</center>
<br>

Notice how **more weight** is put on **positive cases** and and the **most weight** on the **overall probability** of any fractures.

<hr>

**Example:**

Below is a table going through an example of **how the evaluation metric is calculated** for a single case id. This would then be **averaged** across all case id's.

<center>
<img src="https://i.postimg.cc/nh2zCP5T/61415.jpg" width=750>
</center>

Note: it is convention to use the **natural logarithm** (base e) for calculating the log loss. 

NB: it [has been suggested](https://www.kaggle.com/competitions/rsna-2022-cervical-spine-fracture-detection/discussion/344565) that the loss is then also **scaled** by the **sum of weights** on a by-patient basis. So for our example above, the weights were [2,1,1,1,2,1,1,14] which sums to 23 hence the loss would actually be 3.734/23 = 0.162. 


<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>1.3 Code requirements</b></p>
</div>

This is a **code competition**, which means that **submissions** are made through **notebooks**. Furthermore, the submission notebook is subject to these conditions:

* **run-time** (CPU/GPU) **<= 9 hours**
* **internet** access **disabled**
* **external data is allowed**, including pre-trained models
* the submission file must be named **submission.csv**
              
Note that the **test set is hidden**, and will populated when you submit your notebook. 

<hr>

*Competition timeline:*
    
* Start date - 28th July 2022
* Finish date - 27th October 2022

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>1.4 Libraries</b></p>
</div>

In [ ]:
# To read compressed dicom files:

'''Online'''
#! pip install python-gdcm
#! pip install pylibjpeg pylibjpeg-libjpeg pydicom

'''Offline (need to add dataset --> for-pydicom)'''
!pip install -qU ../input/for-pydicom/python_gdcm-3.0.14-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl ../input/for-pydicom/pylibjpeg-1.4.0-py3-none-any.whl --find-links frozen_packages --no-index

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as patches
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.6)
import cv2
import os
from os import listdir
import re
import gc
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tqdm import tqdm
from pprint import pprint
from time import time
import itertools
from skimage import measure 
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import nibabel as nib
from glob import glob
import warnings
#warnings.filterwarnings("ignore", category=DeprecationWarning)
#warnings.filterwarnings("ignore", category=UserWarning)
#warnings.filterwarnings("ignore", category=FutureWarning)

# 2. Data

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>2.1 Data frames</b></p>
</div>


In [ ]:
# Load metadata
train_df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")
train_bbox = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train_bounding_boxes.csv")
test_df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/test.csv")
ss = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/sample_submission.csv")

# Print dataframe shapes
print('train shape:', train_df.shape)
print('train bbox shape:', train_bbox.shape)
print('test shape:', test_df.shape)
print('ss shape:', ss.shape)
print('')

# Show first few entries
train_df.head(3)

> **train.csv** - contains metadata for train_images.
> * StudyInstanceUID - The study ID. There is one unique study ID for each patient scan.
> * patient_overall - The patient level outcome, i.e. if any of the vertebrae are fractured.
> * C[1-7] - Whether the given vertebrae is fractured.

In [ ]:
train_bbox.head(3)

> **train_bounding_boxes.csv** - contains bounding boxes of where fractures occured for a subset of the training set.
> * StudyInstanceUID - The study ID. There is one unique study ID for each patient scan.
> * x - x-coordinate of bounding box bottom left corner
> * y - y-coordinate of bounding box bottom left corner
> * width - width of bounding box
> * height - height of bounding box
> * slice_number - slice number of scan

Note: we only have bounding boxes for a [subset](https://www.kaggle.com/competitions/rsna-2022-cervical-spine-fracture-detection/discussion/343105) of the training set. We'll explore the exact proportion later on.

In [ ]:
test_df.head(3)

> **test.csv** - contains metadata for test_images.
> * row_id - The row ID. This will match the same column in the sample submission file.
> * StudyInstanceUID - The study ID.
> * prediction_type - Which one of the eight target columns needs a prediction in this row.

Note: The full test set will be **populated at inference time**.

In [ ]:
ss.head()

> **sample_submission.csv** - A valid sample submission.
> * row_id - The row ID. See the test.csv for what prediction needs to be filed in that row.
> * fracture - The target column.

# 3. EDA
<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>3.1 Fracture distributions</b></p>
</div>

* The overall target is roughly **balanced** (52/48 split). 
* **C7** has the **highest proportion of fractures** (19%) whereas **C3** has the **lowest** (4%). 
* Several patients have **more than one** fracture.
* If **multiple fractures** occur on a single patient, they tend to occur in vertebrae **close together**, e.g. C4 & C5 as opposed to C1 & C7.

In [ ]:
plt.figure(figsize=(20,6))
plt.subplot(1,2,1)
ax1 = sns.countplot(data=train_df, x='patient_overall')
for container in ax1.containers:
    ax1.bar_label(container)
plt.title('Fractures by patient')
plt.ylim([0,1300])

# Unpivot train_df for plotting
train_melt = pd.melt(train_df, id_vars = ['StudyInstanceUID', 'patient_overall'],
             value_vars = ['C1','C2','C3','C4','C5','C6','C7'],
             var_name="Vertebrae",
             value_name="Fractured")

plt.subplot(1,2,2)
ax2 = sns.countplot(data=train_melt, x='Vertebrae', hue='Fractured')
for container in ax2.containers:
    ax2.bar_label(container)
plt.title('Fractures by vertebrae')
plt.ylim([0,2800])

In [ ]:
plt.figure(figsize=(12,5))
ax = sns.countplot(x = train_df[['C1','C2','C3','C4','C5','C6','C7']].sum(axis=1))
for container in ax.containers:
    ax.bar_label(container)
plt.title('Number of fractures by patient')
plt.xlabel('Number of fractures')
plt.ylim([0,1300])

In [ ]:
# Heatmap of correlations
plt.figure(figsize=(6,5))
sns.heatmap(train_df[['C1','C2','C3','C4','C5','C6','C7']].corr(), cmap='bwr', vmin=-1, vmax=1)
plt.title('Correlations')

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>3.2 Study Id's</b></p>
</div>

The cases in the dataset have **unique id's** like '1.2.826.0.1.3680043.6200'. It turns out only the **number after the last full stop** is important. 

In [ ]:
# Example
train_df['StudyInstanceUID'][0]

In [ ]:
# Find unique numbers in study id's
for i in range(7):
    print(train_df['StudyInstanceUID'].map(lambda x : x.split('.')[i]).unique())

# 4. Train images

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>4.1 What is DICOM?</b></p>
</div>

A **.dcm** file follows the **Digital Imaging and Communications in Medicine** (DICOM) format. It is the standard format used for storing **medical images** and **related metadata**. It dates back to 1983, although it has been revised many times. 

We can use the [pydicom library](https://pydicom.github.io/) to open and explore these files.

In [ ]:
ex_path = "../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.10001/101.dcm"
dcm_example = pydicom.dcmread(ex_path)
dcm_example

The **image data** is stored in an array under **'Pixel Data'**. Everything else is **metadata**.
* The **'Rows'** and **'Columns'** values tell us the **image size**.
* The **'Pixel Spacing'** and **'Slice Thickness'** tell us the **pixel size** and **thickness**.
* The **'Window Center'** and **'Window Width'** give information about the **brightness** and **contrast** of the image respectively.
* The **'Rescale Intercept'** and **'Rescale Slope'** determine the range of pixel values. ([ref](https://stackoverflow.com/questions/10193971/rescale-slope-and-rescale-intercept)).
* **'ImagePositionPatient'** tells us the x, y, and z coordinates of the top left corner of each image in mm
* **InstanceNumber** is the slice number.

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>4.2 Exploring the images</b></p>
</div>

Let's look at the what the **image data** from the dcm files look like.

In [ ]:
# Adapted from https://www.kaggle.com/code/andradaolteanu/rsna-fracture-detection-dicom-images-explore
base_path = "../input/rsna-2022-cervical-spine-fracture-detection"
patient_id = '1.2.826.0.1.3680043.12281'
dcm_paths = glob(f"{base_path}/train_images/{patient_id}/*")
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split(r'(\d+)', text)]
dcm_paths.sort(key=natural_keys)

# Get images
files = [pydicom.dcmread(path) for path in dcm_paths]
images = [apply_voi_lut(file.pixel_array, file) for file in files]

# Plot images
fig, axes = plt.subplots(nrows=3, ncols=6, figsize=(24,12))
fig.suptitle(f'ID: {patient_id}', weight="bold", size=20)

start = 110
for i in range(start,start+18):
    img = images[i]
    file = files[i]
    slice_no = i

    # Plot the image
    x = (i-start) // 6
    y = (i-start) % 6

    axes[x, y].imshow(img, cmap="bone")
    axes[x, y].set_title(f"Slice: {slice_no}", fontsize=14, weight='bold')
    axes[x, y].axis('off')

This is great, but at the moment we don't know which **vertebrae** is being shown in each image. One way to work this out is by using the **segmentations**.

# 5. Segmentations

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>5.1 What is NIfTI? </b></p>
</div>

A **.nii** file follows the **Neuroimaging Informatics Technology Initiative** (NIfTI) format. Compared to the DICOM, NIfTI is **simpler** and **easier** to support. 

To open .nii files we can use the [nibabel library](https://nipy.org/nibabel/gettingstarted.html).

In [ ]:
def load_NIfTI(path):
    nii_example = nib.load(path)

    # Convert to numpy array
    seg = nii_example.get_fdata()
    print(seg.shape)
    
    return seg

In [ ]:
ex_path2 = f"../input/rsna-2022-cervical-spine-fracture-detection/segmentations/{patient_id}.nii"
seg = load_NIfTI(ex_path2)

Each nifti file contains segmentations for **all slices** in a scan. However, we need to be careful about the **orientation** of the segmentations.

> Please be aware that the NIFTI files consist of segmentation in the sagittal plane, while the DICOM files are in the axial plane.

The correct way to deal with this is explained in this [discussion post](https://www.kaggle.com/competitions/rsna-2022-cervical-spine-fracture-detection/discussion/340612). We need to use the following line:

In [ ]:
# Align orientation with images
seg = seg[:, ::-1, ::-1].transpose(2, 1, 0)
seg.shape


<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>5.2 Exploring the masks</b></p>
</div>

In [ ]:
# Plot images
fig, axes = plt.subplots(nrows=3, ncols=6, figsize=(24,12))
fig.suptitle(f'ID: {patient_id}', weight="bold", size=20)

for i in range(start,start+18):
    mask = seg[i]
    slice_no = i

    # Plot the image
    x = (i-start) // 6
    y = (i-start) % 6

    axes[x, y].imshow(mask, cmap='inferno')
    axes[x, y].set_title(f"Slice: {slice_no}", fontsize=14, weight='bold')
    axes[x, y].axis('off')

Compare these with the previous images. These masks give us the **location** of the vertebrae, which is very helpful because we know the fractures can only occur in these regions.

They also tells us which **vertebrae** are in the images. By looking at the unique values in each slice, we find a 0 for the background and another number like 2 for vertebrae C2. 

In [ ]:
np.unique(seg[116])

Unfortunately, we don't have segmentations for the whole train set. Checkout my other [notebook](https://www.kaggle.com/code/samuelcortinhas/extracting-vertebrae-c1-c7), where I exlore how to **extract the vertebrae** for the **whole train set**.

In [ ]:
# Number of cases with masks
seg_paths = glob(f"{base_path}/segmentations/*")
print(f'Number of cases with segmentations: {len(seg_paths)}, ({np.round(100*len(seg_paths)/len(train_df),1)}%)')

An idea to keep in mind is that maybe we could train a **segmentation model** like UNet to predict the segmentation masks for the rest of the train and all of the test images. 

# 6. Extract metadata

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>6.1 Save metadata </b></p>
</div>

This section uses code from Andrada Olteanu.

In [ ]:
# From https://www.kaggle.com/code/andradaolteanu/rsna-fracture-detection-dicom-images-explore
def get_observation_data(path):
    '''
    Get information from the .dcm files
    '''

    dataset = pydicom.read_file(path)
    
    # Dictionary to store the information from the image
    observation_data = {
        "Rows" : dataset.get("Rows"),
        "Columns" : dataset.get("Columns"),
        "SOPInstanceUID" : dataset.get("SOPInstanceUID"),
        "ContentDate" : dataset.get("ContentDate"),
        "SliceThickness" : dataset.get("SliceThickness"),
        "InstanceNumber" : dataset.get("InstanceNumber"),
        "ImagePositionPatient" : dataset.get("ImagePositionPatient"),
        "ImageOrientationPatient" : dataset.get("ImageOrientationPatient"),
    }

    # String columns
    str_columns = ["SOPInstanceUID", "ContentDate", 
                   "SliceThickness", "InstanceNumber"]
    for k in str_columns:
        observation_data[k] = str(dataset.get(k)) if k in dataset else None

    return observation_data

def get_metadata():
    '''
    Retrieves the desired metadata from the .dcm files and saves it into dataframe.
    '''
    
    exceptions = 0
    dicts = []

    for k in tqdm(range(len(train_df))):
        if (k % 100)==0:
            print(f'Iteration: {k}')
            
        dt = train_df.iloc[k, :]

        # Get all .dcm paths for this Instance
        dcm_paths = glob(f"{base_path}/train_images/{dt.StudyInstanceUID}/*")

        for path in dcm_paths:
            try:
                # Get datasets
                dataset = get_observation_data(path)
                dicts.append(dataset)
            except Exception as e:
                exceptions += 1
                continue

    # Convert into df
    meta_train_data = pd.DataFrame(data=dicts, columns=md_example.keys())
    
    # Export information
    meta_train_data.to_csv("meta_train.csv", index=False)
    
    print(f"Metadata created. Number of total fails: {exceptions}.")

In [ ]:
# Example
md_example = get_observation_data(ex_path)
pprint(md_example)

In [ ]:
# Create and save the metadata (~ 2 hours)
#get_metadata()

You can find the dataset containing the metadata [here](https://www.kaggle.com/datasets/samuelcortinhas/rsna-2022-spine-fracture-detection-metadata). (Andrada also has a version containing only the positive patients)

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b>6.2 Explore metadata </b></p>
</div>

In [ ]:
# Read in saved metadata
meta_train = pd.read_csv("../input/rsna-2022-spine-fracture-detection-metadata/meta_train.csv")
meta_train["StudyInstanceUID"] = meta_train["SOPInstanceUID"].apply(lambda x: ".".join(x.split(".")[:-2]))
print('meta_train shape:', meta_train.shape)
meta_train.head(3)

**Image size**

In [ ]:
# Image size
meta_train["ImageSize"] = meta_train["Rows"].astype(str) + " x " + meta_train["Columns"].astype(str)

# Plot image sizes
plt.figure(figsize=(18, 5))
ax = sns.countplot(data=meta_train, x="ImageSize")
for container in ax.containers:
    ax.bar_label(container)
plt.ylim([0,800000])
plt.title('Image sizes in train images', fontsize=25, y=1.02)

Almost all images have size **512x512**. We should therefore **resize** the other images to size 512x512. 

**Content Date**

In [ ]:
# Unique values
meta_train['ContentDate'].unique()

We can drop this feature as it is constant.

In [ ]:
meta_train.drop('ContentDate', axis=1, inplace=True)

**Slice Thickness**

In [ ]:
meta_train["SliceThickness"].unique()

In [ ]:
# Plot slice thickness
plt.figure(figsize=(18, 5))
ax = sns.countplot(data=meta_train, x="SliceThickness")
for container in ax.containers:
    ax.bar_label(container)
ax.set_xticklabels(['0.488','0.500','0.600','0.60...2','0.625','0.664','0.670','0.75','0.800','0.900','1.000'])
plt.ylim([0,420000])
plt.title('Slice thickness distribution', fontsize=25, y=1.02)

**Number of slices per scan**

In [ ]:
# Slice counts
slice_counts = meta_train["StudyInstanceUID"].value_counts().reset_index()
slice_counts.columns = ["StudyInstanceUID", "count"]

# Distribution of slices counts
plt.figure(figsize=(18, 5))
sns.histplot(data=slice_counts, x="count", kde=True, bins=40)
plt.title("Number of slices by scan", size=25, y=1.02)
plt.xlabel("Number of Slices", size = 18)

**Image Position Patient**

In [ ]:
# Extract x, y, z coordinates of position vector
meta_train['ImagePositionPatient_x'] = meta_train['ImagePositionPatient'].apply(lambda x: float(x.replace(',','').replace(']','').replace('[','').split()[0]))
meta_train['ImagePositionPatient_y'] = meta_train['ImagePositionPatient'].apply(lambda x: float(x.replace(',','').replace(']','').replace('[','').split()[1]))
meta_train['ImagePositionPatient_z'] = meta_train['ImagePositionPatient'].apply(lambda x: float(x.replace(',','').replace(']','').replace('[','').split()[2]))

In [ ]:
# Plot position coordinates
plt.figure(figsize=(20,5))
plt.subplot(1,3,1)
sns.histplot(meta_train['ImagePositionPatient_x'])
plt.ylim([0,20000])
plt.title('x-coordinate', fontsize=25, y=1.02)

plt.subplot(1,3,2)
sns.histplot(meta_train['ImagePositionPatient_y'], color='C1')
plt.ylabel('')
plt.ylim([0,20000])
plt.title('y-coordinate', fontsize=25, y=1.02)

plt.subplot(1,3,3)
sns.histplot(meta_train['ImagePositionPatient_z'], color='C2')
plt.ylabel('')
plt.ylim([0,20000])
plt.title('z-coordinate', fontsize=25, y=1.02)

We can use the 'ImagePositionPatient_z' feature to infer the vertebrae present in each slice. It seems non-trivial though - I need to think about this.

**Image Orientation Patient**

This feature tells us coordinates for the position of the patient in the scanner. It could tell us if the images are slightly rotated for example. 

In [ ]:
meta_train['ImageOrientationPatient'].nunique()

In [ ]:
meta_train['ImageOrientationPatient'].unique()[:5]

### Clean meta data

In [ ]:
# Clean metadata
meta_train_clean = meta_train.drop(['SOPInstanceUID','ImagePositionPatient','ImageOrientationPatient','ImageSize'], axis=1)
meta_train_clean.rename(columns={"Rows": "ImageHeight", "Columns": "ImageWidth","InstanceNumber": "Slice"}, inplace=True)
meta_train_clean = meta_train_clean[['StudyInstanceUID','Slice','ImageHeight','ImageWidth','SliceThickness','ImagePositionPatient_x','ImagePositionPatient_y','ImagePositionPatient_z']]
meta_train_clean.sort_values(by=['StudyInstanceUID','Slice'], inplace=True)
meta_train_clean.reset_index(drop=True, inplace=True)

# Export information
#meta_train_clean.to_csv("meta_train_clean.csv", index=False)

# Preview first few columns
meta_train_clean.head(3)

### Get metadata for segmentations



In [ ]:
# Metadata was extracted previously (check out my RSNA dataset)
meta_train = pd.read_csv("../input/rsna-2022-spine-fracture-detection-metadata/meta_train_clean.csv")

# Only select patients with segmentations
meta_seg = meta_train[meta_train['StudyInstanceUID'].isin(seg_df['StudyInstanceUID'])].reset_index(drop=True)
print('meta_seg shape:', meta_seg.shape)
meta_seg.head(3)

The **cleaned version** of this metadata is also in my dataset [here](https://www.kaggle.com/datasets/samuelcortinhas/rsna-2022-spine-fracture-detection-metadata).

### Extract vertebrae from segmentations



In [ ]:
# Initialise targets
targets = ['C1','C2','C3','C4','C5','C6','C7']
meta_seg[targets]=0


In [ ]:
'''
k=0
# Loop over 87 patients with segmentations
for path, UID in zip(seg_df['path'], seg_df['StudyInstanceUID']):
    # Get segmentations for patient
    seg_nib = nib.load(path)
    seg = seg_nib.get_fdata()
    seg = seg[:, ::-1, ::-1].transpose(2, 1, 0) # Align orientation with train images
    num_slices, _, _ = seg.shape
    
    # Loop over slices
    for i in range(num_slices):
        mask = seg[i]
        unique_vals = np.unique(mask)
        
        # Loop over unique values (except 0)
        for j in unique_vals[1:]:
            
            # Ignore thoratic spine etc
            if j <= 7:   
                meta_seg.loc[(meta_seg['StudyInstanceUID']==UID)&(meta_seg['Slice']==i),f'C{int(j)}'] = 1
                
    # Iteration tracker
    if (k%10)==0:
        print(f'Iteration:{k}')
    k+=1

# Save extracted targets
meta_seg.to_csv("meta_segmentation.csv", index=False)
'''

In [ ]:
meta_seg = pd.read_csv('../input/rsna-2022-spine-fracture-detection-metadata/meta_segmentation.csv')
meta_seg.head(3)

#### Examples

In [ ]:
# Example
meta_seg[['StudyInstanceUID','Slice']+targets].iloc[199:204,:]

In [ ]:
# Print example of extracted vertebrae
print('UID:', meta_seg['StudyInstanceUID'].unique()[0])
pd.set_option('display.max_rows', 500)
meta_seg[meta_seg['StudyInstanceUID']==meta_seg['StudyInstanceUID'].unique()[0]].loc[110:340,targets]

Notice how the targets are **monotonic** (C1->C2->C3...) and sometimes **overlap** (e.g. C1 and C2 can appear in the same image).


Thoughts:

* We can use this df to train a model to identify targets for the rest of the train images.
* We can first try building a model using the metadata only (this will be faster) as a baseline.
* Imposing monotonicity will be tricky (maybe we can try to predict the slices where the vertebrae appear and disappear as opposed trying to predict which vertebrae is in every image).
* Then we can then try to use the image data as well for the highest accuracy (this will be more time comsuming).

### Build baseline supervised model



In [ ]:
# Calculate slice ratio (to generalise better)
slice_max_seg = meta_seg.groupby('StudyInstanceUID')['Slice'].max().to_dict()
meta_seg['SliceRatio'] = 0
meta_seg['SliceRatio'] = meta_seg['Slice']/meta_seg['StudyInstanceUID'].map(slice_max_seg)

In [ ]:
features = ['SliceRatio','SliceThickness','ImagePositionPatient_x','ImagePositionPatient_y','ImagePositionPatient_z']

# Features and targets
X = meta_seg[features]
y = meta_seg[targets]

# Train-validation split
X_train, X_valid, y_train, y_valid = train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=0)

In [ ]:
# Classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Score model
print('Classifier average accuracy:', clf.score(X_valid,y_valid))

In [ ]:
# Feature importances
pd.DataFrame({'Feature':features, 'Importance':clf.feature_importances_}).sort_values(by='Importance', ascending=False)

In [ ]:
preds = clf.predict(X_valid)

# Confusion matrices
fig = plt.figure(figsize=(20,10))
for i in range(7):
    cm = confusion_matrix(preds[:,i], y_valid.values[:,i])
    plt.subplot(2,4,i+1)
    CBAR=False
    if (i==3) or (i==6):
        CBAR=True
    sns.heatmap(cm, annot=True, fmt='d', cbar=CBAR, cmap='Blues')
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.title(f'C{i+1}')
fig.tight_layout()

#### Example of predictions



In [ ]:
np.set_printoptions(threshold=np.inf)
clf.predict(X)[110:250,:]

It seems to be preserving monotonicity surprising very well!

**Predict vertebrae numbers on entire train set**

In [ ]:
# Read in metadata for entire train set
meta_train = pd.read_csv('../input/rsna-2022-spine-fracture-detection-metadata/meta_train_clean.csv')

# Calculate slice ratio (to generalise better)
slice_max_train = meta_train.groupby('StudyInstanceUID')['Slice'].max().to_dict()
meta_train['SliceRatio'] = 0
meta_train['SliceRatio'] = meta_train['Slice']/meta_train['StudyInstanceUID'].map(slice_max_train)

# Initialise targets
meta_train[targets]=0

# Predict targets for entire train set
meta_train[targets] = clf.predict(meta_train[features])

# We know images with segmentations have 100% accurate targets so put these back in
meta_train.loc[meta_train['StudyInstanceUID'].isin(meta_seg['StudyInstanceUID']),targets] = meta_seg[targets].values

# Save to csv
meta_train.to_csv('meta_train_with_vertebrae.csv', index=False)

# Preview
meta_train.head(3)

### Plot distributions



In [ ]:
fig = plt.figure(figsize=(20,16))
for i, Cx in enumerate(targets):
    plt.subplot(4,2,i+1)
    sns.histplot(meta_train.groupby('StudyInstanceUID')[Cx].sum())
    plt.title(f'Number of slices: {Cx}')
fig.tight_layout()


# 7. Bounding boxes

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b> 7.1 Box distributions </b></p>
</div>

We are only given bounding boxes for a **subset** of the data. In particular, only **12%** of patients in the train set have any bounding box measurements.

This information is useful in telling us exactly where the fractures have occured. We could consider training an **object localisation** algorithm to provide bounding boxes for the whole train set. 

In [ ]:
print(f'Patients with bounding box measurements: {train_bbox["StudyInstanceUID"].nunique()} ({np.round(100*train_bbox["StudyInstanceUID"].nunique()/len(train_df),1)} %)')

The bounding boxes are only provided for patients with fractures (but not all of them).

In [ ]:
# From https://www.kaggle.com/code/leventelippenszky/rsna-eda-dicom-segmentations-bboxes-3d-plot
train_df_bbox = train_df[train_df["StudyInstanceUID"].isin(train_bbox["StudyInstanceUID"])]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))
sns.countplot(x="patient_overall", data=train_df_bbox, ax=ax1)
ax1.set_title("Fracture overall (patients with bboxes)")

train_df_bbox_melt = pd.melt(train_df_bbox, id_vars=["StudyInstanceUID", "patient_overall"], var_name="cervical_vertebrae", value_name="fracture")
sns.countplot(x="cervical_vertebrae", hue="fracture", data=train_df_bbox_melt, ax=ax2)
ax2.set_title("Fracture of cervical vertebrae (patients with bboxes)")
plt.show()

In [ ]:
plt.figure(figsize=(16,5))
sns.histplot(train_bbox["StudyInstanceUID"].value_counts().values, kde=True, bins=40)
plt.title('Number of slices with bounding boxes per patient')
plt.xlabel('Number of bboxes')

In [ ]:
plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
sns.scatterplot(data=train_bbox, x='x', y='y')
plt.title('Anchor coordinates')

plt.subplot(1,2,2)
sns.scatterplot(data=train_bbox, x='width', y='height')
plt.title('Width and heights')

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b> 7.2 Examples of fractures </b></p>
</div>

In [ ]:
def plot_fracture(slice_num,bbox_id,ax_id1,ax_id2):
    file = pydicom.dcmread(f"{base_path}/train_images/{bbox_id}/{slice_num}.dcm")
    img = apply_voi_lut(file.pixel_array, file)
    info = train_bbox[(train_bbox['StudyInstanceUID']==bbox_id)&(train_bbox['slice_number']==slice_num)]
    rect = patches.Rectangle((float(info.x), float(info.y)), float(info.width), float(info.height), linewidth=3, edgecolor='r', facecolor='none')

    axes[ax_id1,ax_id2].imshow(img, cmap="bone")
    axes[ax_id1,ax_id2].add_patch(rect)
    axes[ax_id1,ax_id2].set_title(f"ID:{bbox_id}, Slice: {slice_num}", fontsize=20, weight='bold',y=1.02)
    axes[ax_id1,ax_id2].axis('off')

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(24,24))
plot_fracture(119,'1.2.826.0.1.3680043.25651',0,0)
plot_fracture(156,'1.2.826.0.1.3680043.23817',0,1)
plot_fracture(325,'1.2.826.0.1.3680043.12031',1,0)
plot_fracture(151,'1.2.826.0.1.3680043.11899',1,1)

# 8. Baseline solution

<div style="color:white;display:fill;
            background-color:#e38e05;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 4px;color:white;"><b> 8.1 Submission </b></p>
</div>

We use the **means** but **scale them up** a bit to take into account the competition metric puts **more weight on positive cases**. (See version 14 of this notebook for a derivation of the scaling function.)

In [ ]:
# Put more weight on positive predictions
def scale_up(q):
    return 2*q/(1+q)

preds = train_df.mean(numeric_only=True).map(scale_up).to_dict()
ss['fractured'] = test_df['prediction_type'].map(preds)
ss.to_csv('submission.csv', index=False)
ss.head(3)

# -1. References

* [RSNA Fracture Detection: DICOM & Images Explore](https://www.kaggle.com/code/andradaolteanu/rsna-fracture-detection-dicom-images-explore) by [Andrada Olteanu](https://www.kaggle.com/andradaolteanu).
* [Fork of great EDA with fix for slice count dist](https://www.kaggle.com/code/kretes/fork-of-great-eda-with-fix-for-slice-count-dist) by [Tomasz Bartczak](https://www.kaggle.com/kretes).
* [What are .DCM and .NII files and how to read them](https://www.kaggle.com/code/datark1/what-are-dcm-and-nii-files-and-how-to-read-them) by [Robert Kwiatkowski](https://www.kaggle.com/datark1).
* [RSNA CSF - Cervical Spine Fracture EDA](https://www.kaggle.com/code/allunia/rsna-csf-cervical-spine-fracture-eda) by [Laura Fink](https://www.kaggle.com/allunia).
* [Explaining Data and Submission in detail](https://www.kaggle.com/competitions/rsna-2022-cervical-spine-fracture-detection/discussion/340612) by [Harshit Sheoran](https://www.kaggle.com/harshitsheoran).
* [Cervical Spine Fracture Detection Quick EDA](https://www.kaggle.com/code/ipythonx/cervical-spine-fracture-detection-quick-eda) by [M.Innat](https://www.kaggle.com/ipythonx).
* [RSNA EDA: DICOM, segmentations, bboxes, 3D plot](https://www.kaggle.com/code/leventelippenszky/rsna-eda-dicom-segmentations-bboxes-3d-plot/notebook) by [_lev_lipinski](https://www.kaggle.com/leventelippenszky).